# Assignment 12

In [Assignment 10](https://github.com/PGE383-HPC-Fall2018/assignment10) we used [Numba](http://numba.pydata.org/) and in [Assignment 11](https://github.com/PGE383-HPC-Fall2018/assignment11) we used [CFFI](https://cffi.readthedocs.io/en/latest/) to speed up the implementation of a two dimensional finite difference solution to the Laplace equation.  In this assignment we will use [SWIG](http://www.swig.org/).  

While all three methods will likely give similar results, they all have thier advantages and disadvantages for the particular use case.  SWIG is particulary useful in wrapping functions from existing C/C++ libraries, especially where several functions may share identical argument signatures.  Additionally, we can use the included [numpy.i](./numpy.i) file to create Python wrappers that are "Numpy aware", meaning that they can take advantage of the fact that Numpy objects know thier shape (e.g. number of rows/columns in a 2D Numpy array) to produce Python wrapper functions that have more simplified argument signatures than the underling C/C++ code.  Finally, although we won't use the capability in this assignment, the same SWIG interface files can be used to generate wrappers for languages in addition to Python, such as Javascript or Perl.

I have written the two functions `iterate` and `iterate_red_black` in C.  These implementation are in the file [iterate.c](iterate.c) and the function declarations are in the header file [iterate.h](iterate.h).  It's your task to write a SWIG interface file [iterate.i](iterate.i) that wraps the functions `iterate` and `iterate_red_black` defined in [iterate.c](iterate.c). The Python module that SWIG generates can then be imported into [laplace.py](laplace.py) and the functions `iterate` and `iterate_red_black` can be called as shown in `swig_solve()` and `swig_solve_red_black()`.  Just uncomment out the import statement and return line.

Because the array `u` is a numpy array, I have included a [numpy.i](numpy.i) SWIG interface file, as well.  This can be used to assist in writing your [iterate.i](iterate.i) file.  You may want to look at the [numpy.i Documentation](https://docs.scipy.org/doc/numpy-1.13.0/reference/swig.interface-file.html) for additional help.  Once you've used used `swig` to generate the interface, you can use the following command to compile the iterface into a dynamic library

```bash
gcc -fPIC -shared iterate*.c -o _iterate.so -I$CONDA_PREFIX/include/python3.6m -I$CONDA_PREFIX/lib/python3.6/site-packages/numpy/core/include
```

The code below can be uncommented and used to test the timing of the speedup of the wrapped functions.

In [1]:
from laplace import LaplaceSolver
solver = LaplaceSolver(nx=20, ny=20)
solver.set_boundary_condtion('top', lambda x,y: 10)
solver.set_boundary_condtion('bottom', lambda x,y: 10)

In [2]:
%timeit solver.solve(quiet=True)

555 µs ± 2.56 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
solver.reset()
%timeit solver.swig_solve(quiet=True)

5.76 µs ± 168 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
solver.reset()
%timeit solver.swig_solve_red_black(quiet=True)

6.14 µs ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
solver.reset()
solver.swig_solve()

Solution converged in 1222 iterations.


In [6]:
solver.reset()
solver.swig_solve_red_black()

Solution converged in 1217 iterations.
